# Benchmarking deep descriptors

Here we will evaluate all descriptors, which are available easily enough, e.g. from kornia or authors github implementation. There will be final comparison table in the end of this notebook.

# SIFT and RootSIFT, [kornia](https://kornia.readthedocs.io/en/latest/feature.html#kornia.feature.SIFTDescriptor) implementation

In [3]:
from IPython.display import clear_output
import torch
import kornia
from brown_phototour_revisited.benchmarking import *
def nice_results_string(desc_name, res_dict):
    if 'liberty' in res_dict:
        lib = f'{(100*res_dict["liberty"]):.2f}'
    else:
        lib = '-----'
    if 'notredame' in res_dict:
        notre = f'{(100*res_dict["notredame"]):.2f}'
    else:
        notre = '-----'
    if 'yosemite' in res_dict:
        yos = f'{(100*res_dict["yosemite"]):.2f}'
    else:
        yos = '-----'
    
    res = f'{desc_name[:20].ljust(20)}   {lib}      {notre}      {yos} '
    return res
TITLE = 'desc_name              liberty   notredame  yosemite'

descs_out_dir = 'data/descriptors'
download_dataset_to = 'data/dataset'
results_dir = 'data/mAP'

full_results_dict = {}
patch_size = 32

In [7]:
desc_name = 'HardNet'
model = kornia.feature.HardNet(True).eval()
models = {'liberty': model}
desc_dict = full_evaluation(models,
                            desc_name,
                            path_to_save_dataset = download_dataset_to,
                            path_to_save_descriptors = descs_out_dir,
                            path_to_save_mAP = results_dir,
                            patch_size = patch_size, 
                            device = torch.device('cuda:0'), 
                       distance='euclidean',
                       backend='pytorch-cuda')
full_results_dict[f'{desc_name} {patch_size}px'] = desc_dict
clear_output()
print (TITLE)
for k, v in full_results_dict.items():
    print (nice_results_string(k,  v['liberty']))


desc_name              liberty   notredame  yosemite
HardNet 32px           -----      61.93      70.64 


In [52]:
desc_name = 'SOSNet'
model = kornia.feature.SOSNet(True).eval()
models = {'liberty': model}
desc_dict = full_evaluation(models,
                            desc_name,
                            path_to_save_dataset = download_dataset_to,
                            path_to_save_descriptors = descs_out_dir,
                            path_to_save_mAP = results_dir,
                            patch_size = patch_size, 
                            device = torch.device('cuda:0'), 
                       distance='euclidean',
                       backend='pytorch-cuda')
full_results_dict[f'{desc_name} {patch_size}px'] = desc_dict
clear_output()
print (TITLE)
for k, v in full_results_dict.items():
    print (nice_results_string(k,  v['3rdparty']))


desc_name              liberty   notredame  yosemite
HardNet 32px           -----      -----      ----- 
TFeat 32px             -----      -----      ----- 
SoftMargin 32px        -----      -----      ----- 
R2D2_center_grayscal   54.98      53.18      61.47 
R2D2_MenCenter_grays   56.25      54.50      63.27 
SOSNet 32px            -----      -----      ----- 


# TFeat


In [8]:
# https://github.com/vbalnt/tfeat/blob/master/tfeat_model.py
import torch
from torch import nn
import torch.nn.functional as F

class TNet(nn.Module):
    """TFeat model definition
    """
    def __init__(self):
        super(TNet, self).__init__()
        self.features = nn.Sequential(
            nn.InstanceNorm2d(1, affine=False),
            nn.Conv2d(1, 32, kernel_size=7),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=6),
            nn.Tanh()
        )
        self.descr = nn.Sequential(
            nn.Linear(64 * 8 * 8, 128),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.descr(x)
        return x

In [11]:
!wget https://github.com/vbalnt/tfeat/raw/master/pretrained-models/tfeat-liberty.params
!wget https://github.com/vbalnt/tfeat/raw/master/pretrained-models/tfeat-yosemite.params
!wget https://github.com/vbalnt/tfeat/raw/master/pretrained-models/tfeat-notredame.params

--2020-09-14 17:38:09--  https://github.com/vbalnt/tfeat/raw/master/pretrained-models/tfeat-liberty.params
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vbalnt/tfeat/master/pretrained-models/tfeat-liberty.params [following]
--2020-09-14 17:38:10--  https://raw.githubusercontent.com/vbalnt/tfeat/master/pretrained-models/tfeat-liberty.params
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2400533 (2.3M) [application/octet-stream]
Saving to: ‘tfeat-liberty.params’

100%[======================================>] 2,400,533   --.-K/s   in 0.08s   

2020-09-14 17:38:17 (30.4 MB/s) - ‘tfeat-liberty.params’ saved [2400533/2400533]

--2020-09-1

In [14]:
desc_name = 'TFeat'
tfeat_lib = TNet()
tfeat_lib.load_state_dict(torch.load('tfeat-liberty.params'))

tfeat_notre = TNet()
tfeat_notre.load_state_dict(torch.load('tfeat-notredame.params'))
                            
tfeat_yos = TNet()
tfeat_yos.load_state_dict(torch.load('tfeat-yosemite.params'))


models = {'liberty': tfeat_lib,
          'yosemite': tfeat_yos,
          'notredame': tfeat_notre}
desc_dict = full_evaluation(models,
                            desc_name,
                            path_to_save_dataset = download_dataset_to,
                            path_to_save_descriptors = descs_out_dir,
                            path_to_save_mAP = results_dir,
                            patch_size = patch_size, 
                            device = torch.device('cuda:0'), 
                       distance='euclidean',
                       backend='pytorch-cuda')
full_results_dict[f'{desc_name} {patch_size}px'] = desc_dict
clear_output()
print (TITLE)
for k, v in full_results_dict.items():
    print (nice_results_string(k,  v['liberty']))

desc_name              liberty   notredame  yosemite
HardNet 32px           -----      61.93      70.64 
TFeat 32px             -----      54.99      65.45 


# Dynamic Soft Margin HardNet

In [15]:
!wget https://github.com/lg-zhang/dynamic-soft-margin-pytorch/raw/master/pretrained/liberty_float/model.state_dict -O dsm_lib.pth
!wget https://github.com/lg-zhang/dynamic-soft-margin-pytorch/raw/master/pretrained/notredame_float/model.state_dict -O dsm_notre.pth
!wget https://github.com/lg-zhang/dynamic-soft-margin-pytorch/raw/master/pretrained/yosemite_float/model.state_dict -O dsm_yos.pth

--2020-09-14 17:53:50--  https://github.com/lg-zhang/dynamic-soft-margin-pytorch/raw/master/pretrained/liberty_float/model.state_dict
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lg-zhang/dynamic-soft-margin-pytorch/master/pretrained/liberty_float/model.state_dict [following]
--2020-09-14 17:53:51--  https://raw.githubusercontent.com/lg-zhang/dynamic-soft-margin-pytorch/master/pretrained/liberty_float/model.state_dict
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5347698 (5.1M) [application/octet-stream]
Saving to: ‘dsm_lib.pth’

100%[======================================>] 5,347,698   27.9MB/s   in 0.2s   

2020-09-14 17:53:59 (

In [16]:
desc_name = 'SoftMargin'
tfeat_lib = kornia.feature.HardNet(False).eval()
tfeat_lib.load_state_dict(torch.load('dsm_lib.pth'))

tfeat_notre = kornia.feature.HardNet(False).eval()
tfeat_notre.load_state_dict(torch.load('dsm_notre.pth'))
                            
tfeat_yos = kornia.feature.HardNet(False).eval()
tfeat_yos.load_state_dict(torch.load('dsm_yos.pth'))


models = {'liberty': tfeat_lib,
          'yosemite': tfeat_yos,
          'notredame': tfeat_notre}
desc_dict = full_evaluation(models,
                            desc_name,
                            path_to_save_dataset = download_dataset_to,
                            path_to_save_descriptors = descs_out_dir,
                            path_to_save_mAP = results_dir,
                            patch_size = patch_size, 
                            device = torch.device('cuda:0'), 
                       distance='euclidean',
                       backend='pytorch-cuda')
full_results_dict[f'{desc_name} {patch_size}px'] = desc_dict
clear_output()
print (TITLE)
for k, v in full_results_dict.items():
    print (nice_results_string(k,  v['liberty']))

desc_name              liberty   notredame  yosemite
HardNet 32px           -----      61.93      70.64 
TFeat 32px             -----      54.99      65.45 
SoftMargin 32px        -----      61.82      69.29 


# HardNetPS

HardNetPS is the HardNet version, trained on the [PS dataset](https://github.com/rmitra/PS-Dataset), which does very well on HPatches, but badly on IMC benchmark.

In [17]:
!wget https://github.com/DagnyT/hardnet/raw/master/pretrained/3rd_party/HardNetPS/HardNetPS.pth

--2020-09-14 18:07:35--  https://github.com/DagnyT/hardnet/raw/master/pretrained/3rd_party/HardNetPS/HardNetPS.pth
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DagnyT/hardnet/master/pretrained/3rd_party/HardNetPS/HardNetPS.pth [following]
--2020-09-14 18:07:35--  https://raw.githubusercontent.com/DagnyT/hardnet/master/pretrained/3rd_party/HardNetPS/HardNetPS.pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5351453 (5.1M) [application/octet-stream]
Saving to: ‘HardNetPS.pth’

100%[======================================>] 5,351,453   30.1MB/s   in 0.2s   

2020-09-14 18:07:44 (30.1 MB/s) - ‘HardNetPS.pth’ saved [5351453/5351453]



In [55]:
class HardNetPS(nn.Module):
    def __init__(self):
        super(HardNetPS, self).__init__()
        self.features = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, padding=1, bias = True),
        nn.BatchNorm2d(32, affine=True),
        nn.ReLU(),
        nn.Conv2d(32, 32, kernel_size=3, padding=1, bias = True),
        nn.BatchNorm2d(32, affine=True),
        nn.ReLU(),
        nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1, bias = True),
        nn.BatchNorm2d(64, affine=True),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=3, padding=1, bias = True),
        nn.BatchNorm2d(64, affine=True),
        nn.ReLU(),
        nn.Conv2d(64, 128, kernel_size=3, stride=2,padding=1, bias = True),
        nn.BatchNorm2d(128, affine=True),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size=3, padding=1, bias = True),
        nn.BatchNorm2d(128, affine=True),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size=8, bias = True)
    )
    def input_norm(self,x):
        flat = x.view(x.size(0), -1)
        mp = torch.mean(flat, dim=1)
        sp = torch.std(flat, dim=1) + 1e-7
        return (x - mp.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand_as(x)) / sp.unsqueeze(-1).unsqueeze(-1).unsqueeze(1).expand_as(x)

    def forward(self, input):
        x_features = self.features(self.input_norm(input))
        x = x_features.view(x_features.size(0), -1)
        return F.normalize(x, p=2, dim=1)


desc_name = 'HardNetPS'
model = HardNetPS().eval()
model.load_state_dict(torch.load('HardNetPS.pth'))
desc_dict = full_evaluation(model,
                            desc_name,
                            path_to_save_dataset = download_dataset_to,
                            path_to_save_descriptors = descs_out_dir,
                            path_to_save_mAP = results_dir,
                            patch_size = patch_size, 
                            device = torch.device('cuda:0'), 
                       distance='euclidean',
                       backend='pytorch-cuda')
full_results_dict[f'{desc_name} {patch_size}px'] = desc_dict
clear_output()
print (TITLE)
for k, v in full_results_dict.items():
    if k == 'HardNetPS':
        print (nice_results_string(k,  v['3rdparty']))
    else:
        print (nice_results_string(k,  v['liberty']))


desc_name              liberty   notredame  yosemite
HardNet 32px           -----      61.93      70.64 
TFeat 32px             -----      54.99      65.45 
SoftMargin 32px        -----      61.82      69.29 
R2D2_center_grayscal   -----      -----      ----- 
R2D2_MenCenter_grays   -----      -----      ----- 
SOSNet 32px            -----      62.09      70.03 
HardNetPS 32px         -----      -----      ----- 


# R2D2

That is NOT fair benchmark for R2D2 for 2 reasons. First, it is dense descriptor, which outputs 32x32 descriptor field for 32x32 patch. We take central pixel, which is kind of reasonable, but not the thing, R2D2 is trained for.
Second, it expects RGB patches and Brown dataset is in grayscale. 
Anyway, let's try!


In [19]:
!wget https://raw.githubusercontent.com/naver/r2d2/master/nets/patchnet.py
!wget https://github.com/naver/r2d2/raw/master/models/r2d2_WASF_N8_big.pt

--2020-09-14 18:09:07--  https://github.com/naver/r2d2/raw/master/models/r2d2_WASF_N8_big.pt
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/naver/r2d2/master/models/r2d2_WASF_N8_big.pt [following]
--2020-09-14 18:09:08--  https://raw.githubusercontent.com/naver/r2d2/master/models/r2d2_WASF_N8_big.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4171550 (4.0M) [application/octet-stream]
Saving to: ‘r2d2_WASF_N8_big.pt’

100%[======================================>] 4,171,550   25.8MB/s   in 0.2s   

2020-09-14 18:09:16 (25.8 MB/s) - ‘r2d2_WASF_N8_big.pt’ saved [4171550/4171550]



In [30]:
from patchnet import *

R2D2 = Quad_L2Net_ConfCFS(mchan=6)
weights = torch.load('r2d2_WASF_N8_big.pt')
print (weights['net'])
weights2 = {}
for k, v in weights['state_dict'].items():
    weights2[k.replace('module.','')] = v
R2D2.load_state_dict(weights2)
R2D2.eval()

Quad_L2Net_ConfCFS(mchan=6)


Quad_L2Net_ConfCFS(
  (ops): ModuleList(
    (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
    (10): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
    (13): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=False, track_run

In [38]:
class R2D2_Center(torch.nn.Module):
    def __init__(self, r2d2):
        super(R2D2_Center, self).__init__()
        self.r2d2 = r2d2
        return
    def forward(self,x):
        orig_out = self.r2d2([x.repeat(1,3,1,1)])
        return orig_out['descriptors'][0][...,16,16]

In [64]:
eval_r2d2 = R2D2_Center(R2D2)

In [65]:
desc_name = 'R2D2_center_grayscale'
desc_dict = full_evaluation(eval_r2d2,
                            desc_name,
                            path_to_save_dataset = download_dataset_to,
                            path_to_save_descriptors = descs_out_dir,
                            path_to_save_mAP = results_dir,
                            patch_size = patch_size, 
                            device = torch.device('cuda:0'), 
                       distance='euclidean',
                       backend='pytorch-cuda')
full_results_dict[f'{desc_name} {patch_size}px'] = desc_dict
clear_output()

Another way to get patch descriptors, is to take the mean over central descriptors. Let's try 3x3 window.

In [67]:
class R2D2_MeanCenter(torch.nn.Module):
    def __init__(self, r2d2):
        super(R2D2_MeanCenter, self).__init__()
        self.r2d2 = r2d2
        return
    def forward(self,x):
        orig_out = self.r2d2([x.repeat(1,3,1,1)])
        return F.normalize(orig_out['descriptors'][0][...,15:18, 15:18].mean(dim=-1).mean(dim=-1), p=2, dim=1)

eval_r2d2 = R2D2_MeanCenter(R2D2)
desc_name = 'R2D2_MenCenter_grayscale'
desc_dict = full_evaluation(eval_r2d2,
                            desc_name,
                            path_to_save_dataset = download_dataset_to,
                            path_to_save_descriptors = descs_out_dir,
                            path_to_save_mAP = results_dir,
                            patch_size = patch_size, 
                            device = torch.device('cuda:0'), 
                       distance='euclidean',
                       backend='pytorch-cuda')
full_results_dict[f'{desc_name} {patch_size}px'] = desc_dict
clear_output()
print (TITLE)
for k, v in full_results_dict.items():
    if k == 'HardNetPS':
        print (nice_results_string(k,  v['3rdparty']))
    elif 'R2D2' in k:
        print (nice_results_string(k,  v['3rdparty']))
    else:
        print (nice_results_string(k,  v['liberty']))

desc_name              liberty   notredame  yosemite
HardNet 32px           -----      61.93      70.64 
TFeat 32px             -----      54.99      65.45 
SoftMargin 32px        -----      61.82      69.29 
R2D2_center_grayscal   54.98      53.18      61.47 
R2D2_MenCenter_grays   56.25      54.50      63.27 
SOSNet 32px            -----      62.09      70.03 
HardNetPS 32px         -----      -----      ----- 


A bit better, but not enough. OK, that was an unfair comparison anyway and R2D2 performed quite decently.

In [78]:
TITLE1 = 'trained on       liberty notredame  liberty yosemite  notredame yosemite'
TITLE2 = 'tested  on           yosemite           notredame            liberty'


def nice_results_3rdparty(desc_name, res_dict):
    if 'liberty' in res_dict:
        lib = f'{(100*res_dict["liberty"]):.2f}'
    else:
        lib = '-----'
    if 'notredame' in res_dict:
        notre = f'{(100*res_dict["notredame"]):.2f}'
    else:
        notre = '-----'
    if 'yosemite' in res_dict:
        yos = f'{(100*res_dict["yosemite"]):.2f}'
    else:
        yos = '-----'
    res = f'{desc_name[:20].ljust(20)}   {yos}              {notre}               {lib} '
    return res

NA = '-----'
def nice_results_Brown(desc_name, res_dict):
    lib_yos, lib_notre, yos_notre, yos_lib, notre_lib, notre_yos = NA,NA,NA,NA,NA,NA
    if 'liberty' in res_dict:
        cr = res_dict['liberty']
        if 'notredame' in cr:
            lib_notre = f'{(100*cr["notredame"]):.2f}'
        else:
            lib_notre = NA
        if 'yosemite' in cr:
            lib_yos = f'{(100*cr["yosemite"]):.2f}'
        else:
            lib_yos = NA
    if 'notredame' in res_dict:
        cr = res_dict['notredame']
        if 'liberty' in cr:
            notre_lib = f'{(100*cr["liberty"]):.2f}'
        else:
            notre_lib = NA
        if 'yosemite' in cr:
            notre_yos = f'{(100*cr["yosemite"]):.2f}'
        else:
            notre_yos = NA    
    if 'yosemite' in res_dict:
        cr = res_dict['yosemite']
        if 'liberty' in cr:
            yos_lib = f'{(100*cr["liberty"]):.2f}'
        else:
            yos_lib = NA
        if 'notredame' in cr:
            yos_notre = f'{(100*cr["notredame"]):.2f}'
        else:
            yos_notre = NA    
    
    res = f'{desc_name[:20].ljust(18)} {lib_yos}  {notre_yos}        {lib_notre}  {yos_notre}        {notre_lib}  {yos_lib}'
    return res

def print_results_table(full_res_dict):
    print (TITLE1)
    print (TITLE2)

    for desc_name, desc_results in full_res_dict.items():
        if '3rdparty' in desc_results:
            #print (desc_results)
            if len(desc_results['3rdparty']) == 3:
                print (nice_results_3rdparty(desc_name, desc_results['3rdparty']))
            else:
                print (nice_results_Brown(desc_name, desc_results))
        else:
            print (nice_results_Brown(desc_name, desc_results))
    print ()
    return



In [79]:
print_results_table(full_results_dict)

trained on       liberty notredame  liberty yosemite  notredame yosemite
tested  on           yosemite           notredame            liberty
HardNet 32px       70.64  -----        61.93  -----        -----  -----
TFeat 32px         65.45  65.77        54.99  54.69        56.55  56.24
SoftMargin 32px    69.29  69.20        61.82  58.61        62.37  60.63
R2D2_center_grayscal   61.47              53.18               54.98 
R2D2_MenCenter_grays   63.27              54.50               56.25 
SOSNet 32px        70.03  -----        62.09  -----        -----  -----
HardNetPS 32px         55.56              49.70               49.12 



In [80]:
non_deep = torch.load('non_deep_res.pt')
print_results_table(non_deep)

trained on       liberty notredame  liberty yosemite  notredame yosemite
tested  on           yosemite           notredame            liberty
Kornia RootSIFT 32px   58.24              49.07               49.65 
Kornia RootSIFT 41px   57.83              48.48               49.01 
Kornia RootSIFT 65px   56.70              47.71               48.09 
Kornia SIFT 32px       58.47              47.76               48.70 
Kornia SIFT 41px       58.14              47.30               48.30 
Kornia SIFT 65px       57.04              46.58               47.59 
OpenCV_SIFT 32px       60.01              49.18               50.95 
OpenCV_SIFT 41px       63.61              50.81               53.20 
OpenCV_SIFT 65px       66.97              52.25               54.87 
OpenCV_LATCH 65px  -----  -----        -----  37.26        -----  39.08
OpenCV_LUCID 32px      20.37              23.08               27.24 
OpenCV_LUCID 65px      20.53              23.31               27.65 
OpenCV_FREAK 65px      46.1

If you use the benchmark, please cite it:

    @misc{BrownRevisited2020,
      title={UBC PhotoTour Revisied},
      author={Mishkin, Dmytro},
      year={2020},
      url = {https://github.com/ducha-aiki/brown_phototour_revisited}
    }